In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

### Parent and Child Variables

Parent variables: Variables which affect other variables
Child Variables: Variables that are affected by other variables.

Variables can be both parent and child. Let's look at an example

In [2]:
import pymc3 as pm

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
C:\Users\nathd\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
with pm.Model() as model:
    parameter = pm.Exponential("poisson_param", 1.0)
    data_generator = pm.Poisson("data_generator", parameter)

In [4]:
with model:
    data_plus_one = data_generator + 1

In [5]:
parameter.tag.test_value

array(0.69314718)

To create different model object with the same name we have to re-run the same code as follows:

In [6]:
with pm.Model() as model:
    theta = pm.Exponential("theta", 2.0)
    data_generator = pm.Poisson("data_generator", theta)

To define a completely different model:

In [7]:
with pm.Model() as ab_testing:
    p_A = pm.Uniform("P(A)", 0, 1)
    p_B = pm.Uniform("P(B)", 0, 1)

All pyMC3 variables have an initial value:

In [8]:
print("parameter.tag.test_value =", parameter.tag.test_value)
print("data_generator.tag.test_value =", data_generator.tag.test_value)
print("data_plus_one.tag.test_value =", data_plus_one.tag.test_value)

parameter.tag.test_value = 0.6931471824645996
data_generator.tag.test_value = 0
data_plus_one.tag.test_value = 1


We can also initialize the variables if we wish:

In [9]:
with pm.Model() as model:
    parameter = pm.Exponential("poisson_param", 1.0, testval=0.5)

print("\nparameter.tag.test_value =", parameter.tag.test_value)


parameter.tag.test_value = 0.5


This is useful if the task at hand requires a better initialization.

 - Stochastic variables: Those which are completely undeterministic, even if you knew the variable components, the output would be random e.g. instances of classes Poisson, DiscreteUniform etc.
 - Deterministic Variables: Those variables whose values are not random if the components are known.



#### Initializing Stochastic Variables

A stochastic or random variable initialization requires a *name* argument and parameters specific to the class.

e.g.: `some_variable = pm.DiscreteUniform("discrete_uni_var", 0, 4)`

#### Deterministic Variables 

Similar to stochastic variables, in order to initialize a deterministic variable, you need to call a deterministic class and pass in the function we desire

`deterministic_variable = pm.Deterministic("deterministic variable", some_function_of_variables)`

Also elementary operations like addition or subtraction will also create a deterministic variable.

In [10]:
with pm.Model() as model:
    lambda_1 = pm.Exponential("lambda_1", 1.0)
    lambda_2 = pm.Exponential("lambda_2", 1.0)
    tau = pm.DiscreteUniform("tau", lower=0, upper=10)

new_deterministic_variable = lambda_1 + lambda_2

However if we want our deterministic variable to be tracked by our sampling then we need to implement the constructor as described before for example consider the case of the average number of texts per day example where we had to define the variable:
$$
\lambda = 
\begin{cases}\lambda_1  & \text{if } t \lt \tau \cr
\lambda_2 & \text{if } t \ge \tau
\end{cases}
$$

In the pymc3 code this looks like:


In [11]:
import numpy as np

n_data_points = 5  # in CH1 we had ~70 data points
idx = np.arange(n_data_points)
with model:
    lambda_ = pm.math.switch(tau >= idx, lambda_1, lambda_2)